In [4]:
import os
import shutil
import random
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_datasets as tfds
import sys

! pip install -q tensorflow-datasets

IMG_SIZE = 64
BATCH_SIZE = 32
EPOCHS = 20
FINE_TUNE_EPOCHS = 40
NUM_CLASSES = 10

MODEL_NAME = 'satellite_resnet_FINETUNED_90acc.h5'

CLEAN_WEIGHTS_NAME = 'satellite_resnet_weights_clean.weights.h5'
SEED = 42

print("Downloading EuroSAT data...")
ds_full = tfds.load('eurosat/rgb', split='train', shuffle_files=True, as_supervised=True, with_info=False)
ds_size = sum(1 for _ in ds_full)

def preprocess(image, label):
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

ds_train = ds_full.take(int(0.8 * ds_size)).map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_val = ds_full.skip(int(0.8 * ds_size)).map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
print("Data loading and splitting complete.")

Data loading and splitting complete.


In [5]:
def build_resnet50_model(input_shape, num_classes):
    conv_base = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    model = Sequential([
        conv_base,
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

model = build_resnet50_model((IMG_SIZE, IMG_SIZE, 3), NUM_CLASSES)
conv_base = model.layers[0]

conv_base.trainable = False
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stopping_base = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint_base = ModelCheckpoint(MODEL_NAME, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

print("\n--- PHASE 1: Training Classification Head (20 Epochs) ---")
model.fit(ds_train, epochs=EPOCHS, validation_data=ds_val, callbacks=[early_stopping_base, checkpoint_base], verbose=1)

conv_base.trainable = True

for layer in conv_base.layers:
    if layer.name.startswith('conv5') or layer.name.startswith('res5'):
        layer.trainable = True
    else:
        layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

early_stopping_ft = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint_ft = ModelCheckpoint(MODEL_NAME, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

print("\n--- PHASE 2: Fine-Tuning ResNet50 (40 Epochs for 90%+ Accuracy) ---")
model.fit(
    ds_train,
    epochs=EPOCHS + FINE_TUNE_EPOCHS,
    initial_epoch=EPOCHS,
    validation_data=ds_val,
    callbacks=[early_stopping_ft, checkpoint_ft],
    verbose=1
)

final_model = tf.keras.models.load_model(MODEL_NAME, compile=False)
final_model.save_weights(CLEAN_WEIGHTS_NAME)

print(f"\nFINAL CLEAN WEIGHTS SAVED: {CLEAN_WEIGHTS_NAME}. Download this file and push to GitHub!")

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step

--- PHASE 1: Training Classification Head (20 Epochs) ---
Epoch 1/20
672/675 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2195 - loss: 2.1372
Epoch 1: val_accuracy improved from -inf to 0.39148, saving model to satellite_resnet_FINETUNED_90acc.h5


675/675 ━━━━━━━━━━━━━━━━━━━━ 31s 28ms/step - accuracy: 0.2198 - loss: 2.1363 - val_accuracy: 0.3915 - val_loss: 1.6732
Epoch 2/20
672/675 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3514 - loss: 1.7388
Epoch 2: val_accuracy improved from 0.39148 to 0.44444, saving model to satellite_resnet_FINETUNED_90acc.h5


675/675 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.3514 - loss: 1.7387 - val_accuracy: 0.4444 - val_loss: 1.5386
Epoch 3/20
672/675 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3882 - loss: 1.6486
Epoch 3: val_accuracy did not improve from 0.44444
675/675 ━━━━━━━━━━━━━━━━━━━━ 12s 18ms/step - accuracy: 0.3882 - loss: 1.6485 - val_accuracy: 0.4180 - val_loss: 1.5087
Epoch 4/20
671/675 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4065 - loss: 1.5908
Epoch 4: val_accuracy improved from 0.44444 to 0.47537, saving model to satellite_resnet_FINETUNED_90acc.h5


675/675 ━━━━━━━━━━━━━━━━━━━━ 12s 18ms/step - accuracy: 0.4065 - loss: 1.5908 - val_accuracy: 0.4754 - val_loss: 1.4312
Epoch 5/20
672/675 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4203 - loss: 1.5621
Epoch 5: val_accuracy improved from 0.47537 to 0.48352, saving model to satellite_resnet_FINETUNED_90acc.h5


675/675 ━━━━━━━━━━━━━━━━━━━━ 12s 18ms/step - accuracy: 0.4203 - loss: 1.5622 - val_accuracy: 0.4835 - val_loss: 1.4064
Epoch 6/20
672/675 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4240 - loss: 1.5455
Epoch 6: val_accuracy improved from 0.48352 to 0.51741, saving model to satellite_resnet_FINETUNED_90acc.h5


...

675/675 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.4417 - loss: 1.4951 - val_accuracy: 0.5467 - val_loss: 1.3097
Epoch 15/20
673/675 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4445 - loss: 1.4895
Epoch 15: val_accuracy did not improve from 0.54667
675/675 ━━━━━━━━━━━━━━━━━━━━ 12s 18ms/step - accuracy: 0.4445 - loss: 1.4895 - val_accuracy: 0.5224 - val_loss: 1.3185
Epoch 16/20
675/675 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4454 - loss: 1.4835
Epoch 16: val_accuracy did not improve from 0.54667
675/675 ━━━━━━━━━━━━━━━━━━━━ 12s 18ms/step - accuracy: 0.4454 - loss: 1.4835 - val_accuracy: 0.5413 - val_loss: 1.2936
Epoch 17/20
675/675 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4569 - loss: 1.4673
Epoch 17: val_accuracy did not improve from 0.54667
675/675 ━━━━━━━━━━━━━━━━━━━━ 12s 18ms/step - accuracy: 0.4569 - loss: 1.4673 - val_accuracy: 0.5426 - val_loss: 1.2977
Epoch 18/20
674/675 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4595 - loss: 1.4537
Epoch 18: val_ac

675/675 ━━━━━━━━━━━━━━━━━━━━ 21s 19ms/step - accuracy: 0.4595 - loss: 1.4537 - val_accuracy: 0.5528 - val_loss: 1.2712
Epoch 19/20
673/675 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4666 - loss: 1.4430
Epoch 19: val_accuracy did not improve from 0.55278
675/675 ━━━━━━━━━━━━━━━━━━━━ 12s 17ms/step - accuracy: 0.4666 - loss: 1.4431 - val_accuracy: 0.5344 - val_loss: 1.2776
Epoch 20/20
672/675 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4596 - loss: 1.4476
Epoch 20: val_accuracy improved from 0.55278 to 0.56019, saving model to satellite_resnet_FINETUNED_90acc.h5


675/675 ━━━━━━━━━━━━━━━━━━━━ 12s 18ms/step - accuracy: 0.4596 - loss: 1.4476 - val_accuracy: 0.5602 - val_loss: 1.2990

--- PHASE 2: Fine-Tuning ResNet50 (40 Epochs for 90%+ Accuracy) ---
Epoch 21/60
675/675 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.3281 - loss: 3.5063
Epoch 21: val_accuracy improved from -inf to 0.45796, saving model to satellite_resnet_FINETUNED_90acc.h5


675/675 ━━━━━━━━━━━━━━━━━━━━ 44s 38ms/step - accuracy: 0.3282 - loss: 3.5045 - val_accuracy: 0.4580 - val_loss: 1.4775
Epoch 22/60
675/675 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5035 - loss: 1.4086
Epoch 22: val_accuracy improved from 0.45796 to 0.51000, saving model to satellite_resnet_FINETUNED_90acc.h5


675/675 ━━━━━━━━━━━━━━━━━━━━ 21s 31ms/step - accuracy: 0.5036 - loss: 1.4086 - val_accuracy: 0.5100 - val_loss: 1.3997
Epoch 23/60
674/675 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.5563 - loss: 1.2299
Epoch 23: val_accuracy did not improve from 0.51000
675/675 ━━━━━━━━━━━━━━━━━━━━ 19s 29ms/step - accuracy: 0.5563 - loss: 1.2298 - val_accuracy: 0.4676 - val_loss: 1.7203
Epoch 24/60
674/675 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.6004 - loss: 1.1191
Epoch 24: val_accuracy did not improve from 0.51000
675/675 ━━━━━━━━━━━━━━━━━━━━ 19s 28ms/step - accuracy: 0.6004 - loss: 1.1190 - val_accuracy: 0.4028 - val_loss: 1.6905
Epoch 25/60
675/675 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.6285 - loss: 1.0468
Epoch 25: val_accuracy did not improve from 0.51000
675/675 ━━━━━━━━━━━━━━━━━━━━ 20s 29ms/step - accuracy: 0.6285 - loss: 1.0468 - val_accuracy: 0.4769 - val_loss: 1.5111
Epoch 26/60
675/675 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.6564 - loss: 0.9716
Epoch 26: val_ac